In [4]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Reload candle data
df_candle = pd.read_csv("acm_usdt_15m.csv", parse_dates=["Open Time"])
df_candle = df_candle[["Open Time", "Open", "High", "Low", "Close"]].astype({"Open": float, "High": float, "Low": float, "Close": float})
df_candle.set_index("Open Time", inplace=True)

# Reload and convert match data
df_match = pd.read_csv("all_acm_matches.csv", parse_dates=["Datetime"])
df_match["Datetime"] = df_match["Datetime"].dt.tz_localize("Asia/Singapore").dt.tz_convert("UTC")
df_match["Datetime"] = df_match["Datetime"].dt.tz_localize(None)

# Filter to only those match times that are in the candle data
valid_matches = df_match[df_match["Datetime"].isin(df_candle.index)]

# Begin plotting
fig, ax = plt.subplots(figsize=(15, 6))

# Plot OHLC as four separate lines
ax.plot(df_candle.index, df_candle["Open"], label="Open", linewidth=0.5)
ax.plot(df_candle.index, df_candle["Close"], label="Close", linewidth=0.5)
ax.plot(df_candle.index, df_candle["High"], label="High", linewidth=0.5)
ax.plot(df_candle.index, df_candle["Low"], label="Low", linewidth=0.5)

# Annotations
for _, row in valid_matches.iterrows():
    color = "green" if row["Result"] == "Win" else "red" if row["Result"] == "Loss" else "blue"
    y = df_candle.loc[row["Datetime"], "Close"]
    marker = "^" if row["Result"] == "Win" else "v"
    ax.plot(row["Datetime"], y, marker=marker, color=color, markersize=10)

# Format
ax.set_title("ACM/USDT 15-min OHLC with Match Result Annotations")
ax.set_ylabel("Price (USDT)")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d\n%H:%M'))
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()

# Save
plot_path = "acm_usdt_ohlc_annotated.png"
plt.savefig(plot_path)
plt.close()

plot_path


'acm_usdt_ohlc_annotated.png'

In [ ]:
# Create plots for latest 5 matches, now showing ±10 day windows and annotating all matches within each window
plot_paths_10day_all_annotations = []

for idx, main_row in latest_matches.iterrows():
    match_time = main_row["Datetime"]
    window_start = match_time - timedelta(days=10)
    window_end = match_time + timedelta(days=10)

    df_window = df_candle[(df_candle.index >= window_start) & (df_candle.index <= window_end)]
    if df_window.empty:
        continue

    # Filter all matches within this window
    matches_in_window = df_match_full[df_match_full["Datetime"].between(window_start, window_end)]

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

    ax1.plot(df_window.index, df_window["Open"], label="Open", linewidth=0.5)
    ax1.plot(df_window.index, df_window["Close"], label="Close", linewidth=0.5)
    ax1.plot(df_window.index, df_window["High"], label="High", linewidth=0.5)
    ax1.plot(df_window.index, df_window["Low"], label="Low", linewidth=0.5)

    # Annotate all matches within the window
    for _, row in matches_in_window.iterrows():
        if row["Datetime"] not in df_window.index:
            continue
        y = df_window.loc[row["Datetime"], "Close"]
        color = "green" if row["Result"] == "Win" else "red" if row["Result"] == "Loss" else "blue"
        marker = "^" if row["Result"] == "Win" else "v"
        label = f"{row['League']} MD{row['Matchday']} ({row['Home/Away']})"
        ax1.plot(row["Datetime"], y, marker=marker, color=color, markersize=10)
        ax1.text(row["Datetime"], y, label, fontsize=8, rotation=90, va='bottom', ha='center')

    ax1.set_title(f"ACM/USDT with All Matches in ±10 Days of {match_time.strftime('%Y-%m-%d %H:%M')}")
    ax1.set_ylabel("Price (USDT)")
    ax1.legend()
    ax1.grid(True)

    ax2.plot(df_window.index, df_window["Volume"], label="Volume", color="gray", linewidth=0.5)
    ax2.set_ylabel("Volume")
    ax2.set_xlabel("Time (UTC)")
    ax2.grid(True)
    ax2.legend()

    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d\n%H:%M'))
    plt.xticks(rotation=45)
    plt.tight_layout()

    plot_file = f"/mnt/data/acm_usdt_match_{match_time.strftime('%Y%m%d_%H%M')}_10days_allmatches.png"
    plt.savefig(plot_file)
    plt.close()
    plot_paths_10day_all_annotations.append(plot_file)

plot_paths_10day_all_annotations
